<a href="https://colab.research.google.com/github/vineelbhatti/WhartonDataScienceComp/blob/main/AnalyzingBalls.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
import pandas as pd
import xgboost
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor
from sklearn.preprocessing import LabelEncoder
from sklearn.metrics import mean_squared_error, r2_score
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


***https://fbref.com/en/comps/22/2023/2023-Major-League-Soccer-Stats***

In [ ]:
mls_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/MLS2023DATA.csv", skiprows=[0], header = None)
mls_data = mls_data.dropna(axis=1, how='all')
mls_data.columns = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/Data/MLS2023DATA.csv', nrows=0).columns
mls_data.head(5)

,Squad,Gls,xG,PK,PKatt,Poss,GA,Ranking,FinalPos,GF1,GA1
0,Atlanta Utd,64,48.7,2,4,55.6,52,6,1,0,0
1,Austin,48,42.5,2,2,51.2,54,12,0,0,0
2,CF Montréal,33,37.8,2,3,48.3,49,10,0,0,0
3,Charlotte,43,37.0,5,5,52.7,50,9,1,0,0
4,Chicago Fire,36,38.2,2,2,46.1,49,13,0,0,0


In [ ]:
NSL_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/NSL_data.csv")
NSL_data = NSL_data.rename(columns={'Gls Conceded': 'GA'})
NSL_data.head(5)

,Team,Conference,Gls,GA,xG,Shots,Corner,PK,PKatt,Poss,RSRating,R1Rating
0,ALB,Western,55,67,42.66,429,167,0,5,0.477059,47.659852,3.536768
1,ANC,Western,60,43,52.67,508,157,0,5,0.482941,57.368580,4.259910
2,AUG,Eastern,62,47,50.42,468,171,3,10,0.554118,62.370847,4.676731
3,BAK,Western,47,51,42.28,457,182,1,5,0.472353,43.635166,3.303508
4,BOI,Western,52,52,50.65,488,167,1,8,0.478235,48.409006,3.811059


***Predicting final position based on rankig and other stats. 0 means don't make playoffs, 1 means lose in first round, 2 means lose in second round, etc. 5 means champions***

In [ ]:
X = mls_data[["Gls", "xG", "PK", "PKatt", "Ranking"]]
Y = mls_data["FinalPos"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=20)

In [ ]:
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

RandomForestRegressor()

In [ ]:
print(rf_model.feature_importances_)

[0.07239959 0.18997467 0.24170886 0.0721261  0.42379079]


In [ ]:
y_predictions = rf_model.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_predictions)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test, y_predictions)
print(f'R-squared: {r2}')

Mean Squared Error: 0.29725555555555566
R-squared: 0.820315671641791


In [ ]:
#mls_data["RFPredFinalPos"] = rf_model.predict(X)
#mls_data.head(28)

***Predicting regular season rankings***

In [ ]:
#mls_data['Gls'] = mls_data['Gls'].div(2).round(0)
#mls_data['xG'] = mls_data['xG'].div(2).round(0)

In [ ]:
X = mls_data[["Gls", "xG", "PK", "PKatt", "Poss"]]
Y = mls_data["Ranking"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=20)

In [ ]:
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

RandomForestRegressor()

In [ ]:
print(rf_model.feature_importances_)

[0.78039485 0.0577858  0.05733779 0.04507744 0.05940413]


In [ ]:
y_predictions = rf_model.predict(X_test)

In [ ]:
mse = mean_squared_error(y_test, y_predictions)
print(f'Mean Squared Error: {mse}')

r2 = r2_score(y_test, y_predictions)
print(f'R-squared: {r2}')

Mean Squared Error: 26.802522222222226
R-squared: -0.3484498757763974


In [ ]:
mls_data["RFPredRanking"] = rf_model.predict(X)
mls_data = mls_data.sort_values(by=['RFPredRanking'], ascending=True)

In [ ]:
#mls_data = mls_data.head(18)
mls_data.head(18)

,Squad,Gls,xG,PK,PKatt,Poss,GA,Ranking,FinalPos,GF1,GA1,RFPredRanking
26,St. Louis,59,42.6,5,5,44.2,43,1,1,0,0,1.73
9,FC Cincinnati,56,56.3,8,9,48.8,37,1,3,2,3,2.13
19,Orlando City,54,42.1,5,5,50.4,38,2,2,0,0,2.60
6,Columbus Crew,63,57.7,7,7,57.1,45,3,5,3,2,2.97
28,Vancouver,51,54.6,6,8,49.0,47,6,1,0,0,3.04
13,Los Angeles FC,53,55.6,7,9,51.2,38,3,4,2,0,3.07
20,Philadelphia,54,49.4,11,11,46.1,38,4,2,0,0,3.19
8,Dynamo FC,51,44.9,6,6,51.6,37,4,3,0,2,3.78
12,LA Galaxy,50,51.2,5,5,55.2,65,13,0,0,0,4.06
16,New England,55,42.5,3,3,49.7,45,5,1,0,0,4.57


In [ ]:
X = NSL_data[["Gls", "xG", "PK", "PKatt", "Poss"]]

In [ ]:
NSL_data["RFPredRanking"] = rf_model.predict(X)
NSL_data = NSL_data.sort_values(by=['RFPredRanking'], ascending=True)
NSL_west = NSL_data[NSL_data["Conference"] == "Western"]
NSL_east = NSL_data[NSL_data["Conference"] == "Eastern"]

In [ ]:
NSL_west.head(9)

,Team,Conference,Gls,GA,xG,Shots,Corner,PK,PKatt,Poss,RSRating,R1Rating,RFPredRanking
21,SFS,Western,59,55,49.39,435,140,1,9,0.559118,55.785750,4.374971,3.13
9,FAR,Western,68,46,53.97,446,185,1,5,0.517941,66.239371,4.680152,3.19
1,ANC,Western,60,43,52.67,508,157,0,5,0.482941,57.368580,4.259910,3.28
0,ALB,Western,55,67,42.66,429,167,0,5,0.477059,47.659852,3.536768,3.36
17,OAK,Western,55,50,42.78,449,161,1,7,0.464118,51.926824,3.613902,3.48
19,REN,Western,53,47,45.63,479,174,1,8,0.542353,51.185046,3.740104,3.57
20,SAS,Western,55,48,44.58,479,162,2,9,0.495588,53.589619,3.794732,3.60
4,BOI,Western,52,52,50.65,488,167,1,8,0.478235,48.409006,3.811059,3.63
24,TAC,Western,50,70,37.15,435,163,1,6,0.498529,43.331189,2.979430,3.90


In [ ]:
NSL_east.head(9)

,Team,Conference,Gls,GA,xG,Shots,Corner,PK,PKatt,Poss,RSRating,R1Rating,RFPredRanking
6,DES,Eastern,56,52,38.80,403,159,2,5,0.531176,54.355064,3.330673,3.07
18,PRO,Eastern,65,47,42.06,452,142,2,8,0.474118,63.886588,4.228227,3.10
14,LRO,Eastern,57,41,51.04,431,166,1,7,0.549412,56.410889,3.941139,3.17
23,SPR,Eastern,64,65,36.31,431,149,1,6,0.434118,58.103776,3.870151,3.25
2,AUG,Eastern,62,47,50.42,468,171,3,10,0.554118,62.370847,4.676731,3.27
25,TOL,Eastern,54,52,41.65,449,178,1,8,0.503529,50.931813,3.434174,3.29
7,DOV,Eastern,62,59,50.08,519,156,1,6,0.447059,56.908607,4.420304,3.33
12,LAR,Eastern,54,53,44.26,426,176,3,7,0.506765,52.687710,3.676056,3.58
13,LEX,Eastern,51,45,45.11,472,174,0,5,0.455882,47.689393,3.394550,3.74


***Predicting by Goals***

In [ ]:
X = mls_data[["Gls", "xG", "PK", "PKatt", "Ranking", "Poss"]]
Y = mls_data["GF1"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=20)
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

RandomForestRegressor()

In [ ]:
print(rf_model.feature_importances_)

[0.07068458 0.62963087 0.07507603 0.12100527 0.02106932 0.08253393]


In [ ]:
y_predictions = rf_model.predict(X_test)

In [ ]:
mls_data["RFPredGF1"] = rf_model.predict(X)
mls_data = mls_data[mls_data["RFPredGF1"] > 0.2]
mls_data.head(10)

,Squad,Gls,xG,PK,PKatt,Poss,GA,Ranking,FinalPos,GF1,GA1,RFPredRanking,RFPredGF1
9,FC Cincinnati,56,56.3,8,9,48.8,37,1,3,2,3,2.13,1.65
6,Columbus Crew,63,57.7,7,7,57.1,45,3,5,3,2,2.97,2.47
28,Vancouver,51,54.6,6,8,49.0,47,6,1,0,0,3.04,1.56
13,Los Angeles FC,53,55.6,7,9,51.2,38,3,4,2,0,3.07,1.65
20,Philadelphia,54,49.4,11,11,46.1,38,4,2,0,0,3.19,0.37
0,Atlanta Utd,64,48.7,2,4,55.6,52,6,1,0,0,5.09,0.21
14,Minnesota Utd,43,53.4,4,5,47.1,50,11,0,0,0,9.91,1.06


***Predicting Ratings in Playoffs***

In [ ]:
playoff_mls_data = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/MLS2023DATA - Playoffs.csv")
playoff_mls_data.head(5)

,Squad,Gls,xG,PK,PKatt,Poss,GA,Ranking,RSRating,R1Rating
0,Atlanta Utd,64,48.7,2,4,55.6,52,6,62.474168,3.668116
1,Columbus Crew,63,57.7,7,7,57.1,45,3,67.801854,4.327160
2,Dynamo FC,51,44.9,6,6,51.6,37,4,55.895857,3.362963
3,FC Cincinnati,56,56.3,8,9,48.8,37,1,62.474671,5.708333
4,FC Dallas,41,37.8,1,1,48.8,33,7,41.364656,2.445833


In [ ]:
X = playoff_mls_data[["Gls", "xG","GA", "PK", "PKatt", "RSRating"]]
Y = playoff_mls_data["R1Rating"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=20)

In [ ]:
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

RandomForestRegressor()

In [ ]:
print(rf_model.feature_importances_)

[0.05686152 0.26720197 0.01976979 0.11983867 0.15516877 0.38115928]


In [ ]:
playoff_mls_data["PredRatingR1"] = rf_model.predict(X)
playoff_mls_data.head(10)

,Squad,Gls,xG,PK,PKatt,Poss,GA,Ranking,RSRating,R1Rating,PredRatingR1
0,Atlanta Utd,64,48.7,2,4,55.6,52,6,62.474168,3.668116,3.862436
1,Columbus Crew,63,57.7,7,7,57.1,45,3,67.801854,4.327160,4.561257
2,Dynamo FC,51,44.9,6,6,51.6,37,4,55.895857,3.362963,3.400496
3,FC Cincinnati,56,56.3,8,9,48.8,37,1,62.474671,5.708333,5.063521
4,FC Dallas,41,37.8,1,1,48.8,33,7,41.364656,2.445833,2.509796
5,Los Angeles FC,53,55.6,7,9,51.2,38,3,58.473237,6.103846,4.310258
6,Nashville,38,41.8,5,5,47.7,31,7,42.479091,2.150000,2.537696
7,New England,55,42.5,3,3,49.7,45,5,55.264118,3.212500,3.303056
8,NY Red Bulls,35,46.3,3,4,46.7,37,8,36.025940,2.458333,3.105212
9,Orlando City,54,42.1,5,5,50.4,38,2,57.722660,4.066667,3.574782


In [ ]:
X = NSL_data[["Gls", "xG", "GA", "PK", "PKatt", "RSRating"]]

In [ ]:
NSL_data["R1Rating"] = rf_model.predict(X)

In [ ]:
NSL_data.head(30)

,Team,Conference,Gls,GA,xG,Shots,Corner,PK,PKatt,Poss,RSRating,R1Rating,RFPredRanking
6,DES,Eastern,56,52,38.80,403,159,2,5,0.531176,54.355064,3.032459,3.07
18,PRO,Eastern,65,47,42.06,452,142,2,8,0.474118,63.886588,4.245991,3.10
21,SFS,Western,59,55,49.39,435,140,1,9,0.559118,55.785750,3.870530,3.13
14,LRO,Eastern,57,41,51.04,431,166,1,7,0.549412,56.410889,3.580727,3.17
9,FAR,Western,68,46,53.97,446,185,1,5,0.517941,66.239371,4.493869,3.19
23,SPR,Eastern,64,65,36.31,431,149,1,6,0.434118,58.103776,3.280872,3.25
2,AUG,Eastern,62,47,50.42,468,171,3,10,0.554118,62.370847,4.296850,3.27
1,ANC,Western,60,43,52.67,508,157,0,5,0.482941,57.368580,3.871656,3.28
25,TOL,Eastern,54,52,41.65,449,178,1,8,0.503529,50.931813,3.201146,3.29
7,DOV,Eastern,62,59,50.08,519,156,1,6,0.447059,56.908607,3.819987,3.33


In [ ]:
NSL_data.to_csv("NewPred1.csv")

In [ ]:
MLS_semis = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/MLS_data - final.csv")
MLS_semis.head(5)

,Squad,Gls,xG,PK,PKatt,Poss,GA,Ranking,RSRating,R1Rating,SemiRating,FinalRating
0,Columbus Crew,63,57.7,7,7,57.1,45,3,67.801854,4.327160,7.240,9.02
1,Dynamo FC,51,44.9,6,6,51.6,37,4,55.895857,3.362963,6.600,NaN
2,FC Cincinnati,56,56.3,8,9,48.8,37,1,62.474671,5.708333,9.700,NaN
3,Los Angeles FC,53,55.6,7,9,51.2,38,3,58.473237,6.103846,6.111,6.11


In [ ]:
X = MLS_semis[["Gls", "xG", "GA", "PK", "PKatt", "RSRating", "R1Rating"]]
Y = MLS_semis["SemiRating"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=20)

In [ ]:
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

RandomForestRegressor()

In [ ]:
MLS_semis["PredRatingSemi"] = rf_model.predict(X)
MLS_semis.head(5)

,Squad,Gls,xG,PK,PKatt,Poss,GA,Ranking,RSRating,R1Rating,SemiRating,FinalRating,PredRatingSemi
0,Columbus Crew,63,57.7,7,7,57.1,45,3,67.801854,4.327160,7.240,9.02,7.97728
1,Dynamo FC,51,44.9,6,6,51.6,37,4,55.895857,3.362963,6.600,NaN,8.83864
2,FC Cincinnati,56,56.3,8,9,48.8,37,1,62.474671,5.708333,9.700,NaN,8.83864
3,Los Angeles FC,53,55.6,7,9,51.2,38,3,58.473237,6.103846,6.111,6.11,7.11592


In [ ]:
NSL_semis = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/NSL_semis.csv")
NSL_semis = NSL_semis.rename(columns={'PredRatingR1': 'R1Rating'})
NSL_semis.head(5)

,Unnamed: 0,Team,Conference,Gls,GA,xG,Shots,Corner,PK,PKatt,Poss,RSRating,RFPredRanking,R1Rating
0,2,AUG,Eastern,62,47,50.42,468,171,3,10,0.554118,71.770847,3.10,4.273932
1,7,DOV,Eastern,62,59,50.08,519,156,1,6,0.447059,68.708607,3.13,3.825156
2,17,OAK,Western,55,50,42.78,449,161,1,7,0.464118,61.926824,3.36,3.331621
3,10,FOR,Western,54,63,50.83,409,172,0,1,0.551765,60.580012,4.27,3.283411


In [ ]:
X = NSL_semis[["Gls", "xG", "GA", "PK", "PKatt", "RSRating", "R1Rating"]]

In [ ]:
NSL_semis["SemiRating"] = rf_model.predict(X)
NSL_semis.head(5)

,Unnamed: 0,Team,Conference,Gls,GA,xG,Shots,Corner,PK,PKatt,Poss,RSRating,RFPredRanking,R1Rating,SemiRating
0,2,AUG,Eastern,62,47,50.42,468,171,3,10,0.554118,71.770847,3.10,4.273932,7.97728
1,7,DOV,Eastern,62,59,50.08,519,156,1,6,0.447059,68.708607,3.13,3.825156,7.97728
2,17,OAK,Western,55,50,42.78,449,161,1,7,0.464118,61.926824,3.36,3.331621,7.97728
3,10,FOR,Western,54,63,50.83,409,172,0,1,0.551765,60.580012,4.27,3.283411,7.97728


In [ ]:
MLS_finals = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/MLS_finals.csv")
MLS_finals.head(5)

,Squad,Gls,xG,PK,PKatt,Poss,GA,Ranking,RSRating,R1Rating,SemiRating,FinalRating
0,Columbus Crew,63,57.7,7,7,57.1,45,3,76.801854,4.727160,7.6375,9.22
1,Los Angeles FC,53,55.6,7,9,51.2,38,3,66.073237,6.303846,6.1110,6.11


In [ ]:
X = MLS_finals[["Gls", "xG", "PK", "PKatt", "RSRating", "R1Rating"]]
Y = MLS_finals["FinalRating"]

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.3, random_state=20)

In [ ]:
rf_model = RandomForestRegressor()
rf_model.fit(X_train, y_train)

RandomForestRegressor()

In [ ]:
MLS_finals["PredRatingFinals"] = rf_model.predict(X)
MLS_finals.head(5)

,Squad,Gls,xG,PK,PKatt,Poss,GA,Ranking,RSRating,R1Rating,SemiRating,FinalRating,PredRatingFinals
0,Columbus Crew,63,57.7,7,7,57.1,45,3,76.801854,4.727160,7.6375,9.22,6.11
1,Los Angeles FC,53,55.6,7,9,51.2,38,3,66.073237,6.303846,6.1110,6.11,6.11


In [ ]:
NSL_finals = pd.read_csv("/content/drive/MyDrive/Colab Notebooks/Data/NSL_finals.csv")
NSL_finals = NSL_finals.rename(columns={'PredRatingR1': 'R1Rating'})
NSL_finals.head(5)

,Unnamed: 0,Team,Conference,Gls,xG,Shots,Corner,PK,PKatt,Poss,RSRating,RFPredRanking,R1Rating
0,2,AUG,Eastern,62,50.42,468,171,3,10,0.554118,71.770847,3.10,4.809537
1,7,DOV,Eastern,62,50.08,519,156,1,6,0.447059,68.708607,3.13,4.448757


In [ ]:
X = NSL_finals[["Gls", "xG", "PK", "PKatt", "RSRating", "R1Rating"]]

In [ ]:
NSL_finals["FinalRating"] = rf_model.predict(X)
NSL_finals.head(5)

,Unnamed: 0,Team,Conference,Gls,xG,Shots,Corner,PK,PKatt,Poss,RSRating,RFPredRanking,R1Rating,FinalRating
0,2,AUG,Eastern,62,50.42,468,171,3,10,0.554118,71.770847,3.10,4.809537,6.11
1,7,DOV,Eastern,62,50.08,519,156,1,6,0.447059,68.708607,3.13,4.448757,6.11
